# Load Freely Moving Mice Motion Marker .npy file

In [10]:
import numpy as np

mice = '20250310_VN_harmaline'
path = f'/data1/zhangyuhao/xinchao_data/NP2/{mice}/Marker/'
raw_data = np.load(path + 'motion_marker.npy')
raw_data = raw_data[0]
print(raw_data)
print("最大值：", np.max(raw_data))
print("最小值：", np.min(raw_data))


[7.62939453 8.54492188 5.79833984 ... 5.95092773 6.25610352 5.34057617]
最大值： 4999.847412109375
最小值： -19.073486328125


# Dectect Motion times

In [12]:
def detect_edges_with_binarization(signal, sampling_rate, threshold=2470):
    """
    通过二值化检测信号的上升沿和下降沿时刻

    参数:
        signal: 输入的一维信号数组
        sampling_rate: 采样率 (单位: Hz)
        threshold: 二值化阈值，默认为低电平和高电平的中间值
    
    返回:
        rise_times: 上升沿时刻数组 (单位: 秒)
        fall_times: 下降沿时刻数组 (单位: 秒)
    """
    # 步骤1: 二值化信号（低电平为0，高电平为1）
    binary_signal = np.where(signal >= threshold, 1, 0)
    
    # 步骤2: 检测跳变（通过相邻元素的差值）
    diff_binary = np.diff(binary_signal)
    
    # 步骤3: 提取跳变位置
    # 上升沿：0→1跳变（差值为+1）
    rise_indices = np.where(diff_binary == 1)[0]
    # 下降沿：1→0跳变（差值为-1）
    fall_indices = np.where(diff_binary == -1)[0]
    
    # 步骤4: 转换为时间（补偿diff导致的索引偏移）
    rise_times = (rise_indices + 1) / sampling_rate
    fall_times = (fall_indices + 1) / sampling_rate
    
    return rise_times, fall_times

sampling_rate = 10593.2   # 采样率
threshold = 2470          # 根据高低电平的中间值设定

rise_times, fall_times = detect_edges_with_binarization(raw_data, sampling_rate, threshold)

In [13]:
# 合并并排序
all_edges = np.concatenate([rise_times, fall_times])
ordered_edges = np.sort(all_edges)

In [14]:
np.save(path + 'rise_times.npy', rise_times)
np.save(path + 'fall_times.npy', fall_times)
np.save(path + 'run_stop_times_evenRun_odd_Stop.npy', ordered_edges)